# 0.0 Import Libraries

In [ ]:
pip install spectral mat73  einops

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sys
import os
import math

from einops import rearrange
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from scipy import io
import torch.utils.data
import scipy.io as sio
import mat73
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1.0 Upload Data

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#! ls '/content/drive/MyDrive/A02_RemoteSensingData/TrentoDataSet/'

In [ ]:
# path
#path ='/content/drive/MyDrive/A02_RemoteSensingData/TrentoDataSet/'

In [ ]:
# Loader HSI TR_map_2018
trento_data=sio.loadmat('trento_data.mat')['HSI_data']
trento_data.shape

(166, 600, 63)

In [ ]:
# Loader HSI TR_map_2018
trento_data=sio.loadmat('trento_data.mat')

# Extract the HSI_data and LiDAR_data arrays
trento_hsi_data = trento_data['HSI_data']
trento_lidar_data = trento_data['LiDAR_data']

# Reshape the data
trento_lidar_data = np.reshape(trento_lidar_data, (166, 600, 1))

trento_gt=trento_data['ground']
print('trento_hsi_data shape:', trento_hsi_data.shape)
print('trento_lidar_data shape:', trento_lidar_data.shape)
print('trento_gt shape:', trento_gt.shape)

# Load HSITrSet
trento_hsi_trset=sio.loadmat('HSI_TrSet.mat')['HSI_TrSet']
trento_trlabel=sio.loadmat('TrLabel.mat')['TrLabel']
print('trento_hsi_trset shape:', trento_hsi_trset.shape)
print('trento_trlabel shape:', trento_trlabel.shape)

#Load HSITeset
trento_hsi_teset=sio.loadmat('HSI_TeSet.mat')['HSI_TeSet']
trento_telabel=sio.loadmat('TeLabel.mat')['TeLabel']
print('trento_hsi_teset shape:', trento_hsi_teset.shape)
print('trento_telabel shape:', trento_telabel.shape)

trento_hsi_data shape: (166, 600, 63)
trento_lidar_data shape: (166, 600, 1)
trento_gt shape: (166, 600)
trento_hsi_trset shape: (2832, 144)
trento_trlabel shape: (2832, 1)
trento_hsi_teset shape: (12197, 144)
trento_telabel shape: (12197, 1)


In [ ]:
# 2.1 Define the class information
class_info = [
    (1, "Apple trees", 129, 3905, 4034),
    (2, "Buildings ", 125, 2778, 2903),
    (3, "Ground", 105, 374, 479),
    (4, "Wood", 154, 8969, 9123),
    (5, "Vineyard", 184, 10317, 10501),  # Corrected this line
    (6, "Roads", 122, 3052, 3174)
]

class_dict = {class_number: {"class_name": class_name, "training": training, "test": test, "samples": samples} for class_number, class_name, training, test, samples in class_info}

for class_number, class_info in class_dict.items():
    print(f"Class {class_number}: {class_info['class_name']}")
    print(f"Training Samples: {class_info['training']}")
    print(f"Test Samples: {class_info['test']}")
    print(f"Total Samples: {class_info['samples']}")
    print()

print(class_dict)


Class 1: Apple trees
Training Samples: 129
Test Samples: 3905
Total Samples: 4034

Class 2: Buildings 
Training Samples: 125
Test Samples: 2778
Total Samples: 2903

Class 3: Ground
Training Samples: 105
Test Samples: 374
Total Samples: 479

Class 4: Wood
Training Samples: 154
Test Samples: 8969
Total Samples: 9123

Class 5: Vineyard
Training Samples: 184
Test Samples: 10317
Total Samples: 10501

Class 6: Roads
Training Samples: 122
Test Samples: 3052
Total Samples: 3174

{1: {'class_name': 'Apple trees', 'training': 129, 'test': 3905, 'samples': 4034}, 2: {'class_name': 'Buildings ', 'training': 125, 'test': 2778, 'samples': 2903}, 3: {'class_name': 'Ground', 'training': 105, 'test': 374, 'samples': 479}, 4: {'class_name': 'Wood', 'training': 154, 'test': 8969, 'samples': 9123}, 5: {'class_name': 'Vineyard', 'training': 184, 'test': 10317, 'samples': 10501}, 6: {'class_name': 'Roads', 'training': 122, 'test': 3052, 'samples': 3174}}


# 2.0 Data Preprocessing & Dataloader Preparation

### 2.1  Samples Extraction

In [ ]:

# # Create a mask with all class labels
# mask = np.copy(trento_gt)

# # Set the background class to 0
# mask[mask == 0] = 0

# Define patch size and stride
patch_size = 9
stride = 1

# Create an empty list to store patches and labels
hsi_samples = []
lidar_samples = []
labels = []

# Initialize a dictionary to store class count
class_count = {i: 0 for i in class_dict.keys()}

# Function to check if all classes have the required number of samples
def all_classes_completed(class_count, class_dict):
    return all(class_count[class_num] == class_dict[class_num]["samples"] for class_num in class_dict.keys())

while not all_classes_completed(class_count, class_dict):
    # Loop through the ground truth data
    for label in class_dict.keys():
        # Get the coordinates of the ground truth pixels
        #coords = np.argwhere((trento_gt == label) & (mask > 0))
        coords = np.argwhere((trento_gt == label) )

        # Shuffle the coordinates to randomize the patch extraction
        np.random.shuffle(coords)

        for coord in coords:
            i, j = coord
            # Calculate the patch indices
            i_start, i_end = i - patch_size // 2, i + patch_size // 2 + 1
            j_start, j_end = j - patch_size // 2, j + patch_size // 2 + 1

            # Check if the indices are within the bounds of the HSI data
            if i_start >= 0 and i_end <= trento_hsi_data.shape[0] and j_start >= 0 and j_end <= trento_hsi_data.shape[1]:
                # Extract the patch
                hsi_patch = trento_hsi_data[i_start:i_end, j_start:j_end, :]

                # Extract the LiDAR patch
                lidar_patch = trento_lidar_data[i_start:i_end, j_start:j_end]

                # If the class count is less than the required samples
                if class_count[label] < class_dict[label]["samples"]:
                    # Append the patch and its label to the list
                    hsi_samples.append(hsi_patch)
                    lidar_samples.append(lidar_patch)
                    labels.append(label)
                    class_count[label] += 1

                    # If all classes have the required number of samples, exit the loop
                    if all_classes_completed(class_count, class_dict):
                        break

# Convert the list of patches and labels into arrays
hsi_samples = np.array(hsi_samples)
lidar_samples = np.array(lidar_samples)
labels = np.array(labels)
print('hsi_samples shape:', hsi_samples.shape)
print('lidar_samples shape:', lidar_samples.shape)
print('labels shape:', labels.shape)


hsi_samples shape: (30214, 9, 9, 63)
lidar_samples shape: (30214, 9, 9, 1)
labels shape: (30214,)


### 2.2 Training samples extraction

In [ ]:
from sklearn.model_selection import train_test_split

# Initialize empty lists to store the train and test samples
hsi_train = []
hsi_test = []
lidar_train = []
lidar_test = []
y_train = []
y_test = []

# Split the data for each class
for class_number, class_info in class_dict.items():
    # Get the indices of the samples for this class
    class_indices = np.where(labels == class_number)[0]

    # Shuffle the class indices
    np.random.shuffle(class_indices)

    # Split the class indices into training and test indices
    train_indices = class_indices[:class_info["training"]]
    test_indices = class_indices[class_info["training"]:class_info["samples"]]

    # Add the selected training samples to the train lists
    hsi_train.append(hsi_samples[train_indices])
    lidar_train.append(lidar_samples[train_indices])
    y_train.append(labels[train_indices])

    # Add the remaining samples to the test lists
    hsi_test.append(hsi_samples[test_indices])
    lidar_test.append(lidar_samples[test_indices])
    y_test.append(labels[test_indices])

# Concatenate the train and test lists to create the train and test arrays
hsi_train = np.concatenate(hsi_train)
hsi_test = np.concatenate(hsi_test)
lidar_train = np.concatenate(lidar_train)
lidar_test = np.concatenate(lidar_test)
y_train = np.concatenate(y_train)
y_test = np.concatenate(y_test)

# Print the shapes of the train and test arrays
print('hsi_train shape:', hsi_train.shape)
print('hsi_test shape:', hsi_test.shape)
print('lidar_train shape:', lidar_train.shape)
print('lidar_test shape:', lidar_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)


hsi_train shape: (819, 9, 9, 63)
hsi_test shape: (29395, 9, 9, 63)
lidar_train shape: (819, 9, 9, 1)
lidar_test shape: (29395, 9, 9, 1)
y_train shape: (819,)
y_test shape: (29395,)


In [ ]:
# Normalize channel 1(Height) of the train data
ch1 = hsi_train[:, :, :, 0]
pmin = np.amin(ch1)
pmax = np.amax(ch1)
ch1 = (ch1-pmin) / (pmax- pmin)

# Normalize channel 2(Intensity) of the train data
ch2 = hsi_train[:, :, :, 1]
pmin1 = np.amin(ch2)
pmax1 = np.amax(ch2)
ch2 = (ch2-pmin1) / (pmax1- pmin1)

hsi_train[:,:,:,0] = ch1
hsi_train[:,:,:,1] = ch2

# Normalize channel 1(Height) the test data
ch3 = hsi_test[:, :, :, 0]
ch3 = (ch3-pmin) / (pmax- pmin)

# Normalize channel 2(Intensity) of the test data
ch4 = hsi_test[:, :, :, 1]
ch4 = (ch4-pmin1) / (pmax1- pmin1)

hsi_test[:,:,:,0] = ch3
hsi_test[:,:,:,1] = ch4

print('ch1 shape:',ch1.shape)
print('ch2 shape:',ch2.shape)
print('ch3 shape:',ch3.shape)
print('ch4 shape:',ch4.shape)

ch1 shape: (819, 9, 9)
ch2 shape: (819, 9, 9)
ch3 shape: (29395, 9, 9)
ch4 shape: (29395, 9, 9)


In [ ]:
import numpy as np
from scipy.ndimage import rotate

def augment_training_data(hsi_training_data, lidar_training_data, training_labels, rotations=[45, 90, 135], flip_up_down=True, flip_left_right=True):
    augmented_hsi = []
    augmented_lidar = []
    augmented_labels = []

    for hsi, lidar, label in zip(hsi_training_data, lidar_training_data, training_labels):
        # Original data
        augmented_hsi.append(hsi)
        augmented_lidar.append(lidar)
        augmented_labels.append(label)

        # Rotations
        for angle in rotations:
            hsi_rotated = rotate(hsi, angle, axes=(0, 1), reshape=False, mode='nearest')
            lidar_rotated = rotate(lidar, angle, axes=(0, 1), reshape=False, mode='nearest')

            augmented_hsi.append(hsi_rotated)
            augmented_lidar.append(lidar_rotated)
            augmented_labels.append(label)

        # Flip up-down
        if flip_up_down:
            hsi_flipped_ud = np.flipud(hsi)
            lidar_flipped_ud = np.flipud(lidar)

            augmented_hsi.append(hsi_flipped_ud)
            augmented_lidar.append(lidar_flipped_ud)
            augmented_labels.append(label)

        # Flip left-right
        if flip_left_right:
            hsi_flipped_lr = np.fliplr(hsi)
            lidar_flipped_lr = np.fliplr(lidar)

            augmented_hsi.append(hsi_flipped_lr)
            augmented_lidar.append(lidar_flipped_lr)
            augmented_labels.append(label)

    return np.array(augmented_hsi), np.array(augmented_lidar), np.array(augmented_labels)

# Augmenting the training samples
augmented_hsi_training_samples, augmented_lidar_training_samples, augmented_training_labels = augment_training_data(hsi_train, lidar_train, y_train)

# Print shapes to verify the augmented training data
print('Augmented HSI training samples shape:', augmented_hsi_training_samples.shape)
print('Augmented LiDAR training samples shape:', augmented_lidar_training_samples.shape)
print('Augmented training labels shape:', augmented_training_labels.shape)

Augmented HSI training samples shape: (4914, 9, 9, 63)
Augmented LiDAR training samples shape: (4914, 9, 9, 1)
Augmented training labels shape: (4914,)


In [ ]:
hsi_train=augmented_hsi_training_samples
lidar_train=augmented_lidar_training_samples
y_train=augmented_training_labels
print('hsi_train_samples shape:', hsi_train.shape)
print('lidar_train_samples shape:', lidar_train.shape)
print('train_labels shape:', y_train.shape)

hsi_train_samples shape: (4914, 9, 9, 63)
lidar_train_samples shape: (4914, 9, 9, 1)
train_labels shape: (4914,)


In [ ]:
print('hsi_test_samples shape:', hsi_test.shape)
print('lidar_test_samples shape:', lidar_test.shape)
print('y_test shape:', y_test.shape)

hsi_test_samples shape: (29395, 9, 9, 63)
lidar_test_samples shape: (29395, 9, 9, 1)
y_test shape: (29395,)


In [ ]:
from tensorflow.keras.utils import to_categorical

# One hot encoding of labels
# Substract 1 from labels
y_train_adj = augmented_training_labels - 1
y_test_adj = y_test - 1

# One hot encoding of labels
y_train = to_categorical(y_train_adj, num_classes = 6, dtype ="int32")
y_test = to_categorical(y_test_adj, num_classes = 6, dtype ="int32")

print('y_train.shape:',y_train.shape)
print('y_test.shape:',y_test.shape)

2023-12-29 09:26:51.393522: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


y_train.shape: (4914, 6)
y_test.shape: (29395, 6)


In [ ]:
# from tensorflow.keras.utils import to_categorical

# # One hot encoding of labels
# # Substract 1 from labels
# y_train_adj = y_train - 1
# y_test_adj = y_test - 1

# # One hot encoding of labels
# y_train = to_categorical(y_train_adj, num_classes = 6, dtype ="int32")
# y_test = to_categorical(y_test_adj, num_classes = 6, dtype ="int32")

# print('y_train.shape:',y_train.shape)
# print('y_test.shape:',y_test.shape)

### 2.3 Test samples extracting

#3.0 Model Building

### 3.1 Configuration

In [ ]:
# 3.1 Configuration
class Config:
    def __init__(self,in_channels,num_patches,kernel_size,patch_size,emb_size, dim,depth,heads,dim_head,mlp_dim,num_classes,dropout,pos_emb_size,class_emb_size,stride, ):
        self.in_channels = in_channels
        self.num_patches = num_patches
        self.kernel_size = kernel_size
        self.patch_size = patch_size
        self.emb_size = emb_size
        self.dim = dim
        self.depth = depth
        self.heads = heads
        self.dim_head = dim_head
        self.mlp_dim = mlp_dim
        self.num_classes = num_classes
        self.dropout = dropout
        self.pos_emb_size = pos_emb_size
        self.class_emb_size = class_emb_size
        self.stride = stride



### 3.2 EmbeddingPatches

In [ ]:


class PatchEmbedding(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.proj = nn.Conv2d(
            config.in_channels,
            config.emb_size,
            kernel_size=config.patch_size,
            stride=config.stride,
        )
        self.pos_embedding = nn.Parameter(torch.zeros(1, config.num_patches + 1, config.emb_size))
        self.cls_token = nn.Parameter(torch.zeros(1, 1, config.emb_size))

    def forward(self, x):
        B, C, H, W = x.shape
        x = self.proj(x)
        x = x.flatten(2).transpose(1, 2)
        x = torch.cat((self.cls_token.expand(B, -1, -1), x), dim=1)
        x = x + self.pos_embedding[:, :x.size(1)]
        return x


### 3.2 Optianl Adding bandoutput

In [ ]:
class CrossAttention(nn.Module):
    def __init__(self, lidar_config, hsi_config):
        super(CrossAttention, self).__init__()

        # Define module parameters
        self.dim_head = lidar_config.dim_head
        self.num_patches=hsi_config.num_patches
        self.num_heads = lidar_config.heads
        self.sqrt_dim_head = math.sqrt(self.dim_head)

        # Define linear layers for transforming Q, K, and V
        self.to_q = nn.Linear(lidar_config.dim, self.dim_head * self.num_heads, bias=False)
        self.to_k = nn.Linear(hsi_config.dim, self.dim_head * self.num_heads, bias=False)
        self.to_v = nn.Linear(lidar_config.dim, self.dim_head * self.num_heads, bias=False)

        self.to_out = nn.Linear( self.num_heads* self.dim_head,self.num_patches )  # added


    def forward(self, lidar, hsi):
        B, N_lidar, _ = lidar.size()
        _, N_hsi, _ = hsi.size()

        outputs = []
        attn_scores = []  # List to store attention scores

       # Iterate over lidar and hsi patches
        for i in range(1, N_lidar):
        #for i in range(N_lidar):

            lidar_patch = lidar[:, i].unsqueeze(1)  # Add a dimension for number of patches
            for j in range(1, N_hsi):
            #for j in range(N_hsi):

                hsi_patch = hsi[:, j].unsqueeze(1)  # Add a dimension for number of patches
                Q = self.to_q(lidar_patch)
                K = self.to_k(hsi_patch)
                V = self.to_v(lidar_patch)

                Q = Q / self.sqrt_dim_head
                attn_weights = F.softmax(Q @ K.transpose(-2, -1), dim=-1)

                attn_output = attn_weights @ V
                attn_score = self.to_out(attn_output)  # added
                outputs.append(attn_output)
                attn_scores.append(attn_score)  # Store the attention scores

         # Concatenate all the outputs
        output = torch.cat(outputs, dim=1)
        attn_scores = torch.cat(attn_scores, dim=1)  # Concatenate all the attention scores

        return output, attn_scores  # Return both output and attention scores


In [ ]:
class CrossAttentionModel(nn.Module):
    def __init__(self, hsi_config, lidar_config):
        super().__init__()
        self.hsi_patch_embedding = PatchEmbedding(hsi_config)
        self.lidar_patch_embedding = PatchEmbedding(lidar_config)
        self.cross_attention = CrossAttention(lidar_config, hsi_config)

    def forward(self, lidar_data, hsi_data):
        # Apply PatchEmbedding
        lidar_emb = self.lidar_patch_embedding(lidar_data)
        hsi_emb = self.hsi_patch_embedding(hsi_data)

        # Apply CrossAttention
        output, attn_scores = self.cross_attention(lidar_emb, hsi_emb)

        return output, attn_scores  # Return both output and attention scores


In [ ]:
#3.1.1 Parameters Setting
# Hsi configuration
hsi_config = Config(
    in_channels=81,  # Each sample covers 144 bands
    num_patches=63,  # 25*1*144 bands are grouped into 3600 groups
    kernel_size=1,  # Adjusted to match new patch size
    patch_size=1,  # Adjusted to match new patch size (5*5, 144/24=6)
    emb_size=128,  # Embedding size, this can be changed
    dim=128,  # Dimension of the transformer, this can be changed
    depth=6,  # Number of transformer layers, this can be changed
    heads=8,  # Number of attention heads, this can be changed
    dim_head=64,  # Dimension of each attention head, this can be changed
    mlp_dim=256,  # Dimension of the MLP layer, this can be changed
    num_classes=15,  # Number of classes, this can be changed
    dropout=0.4,  # Dropout rate, this can be changed
    pos_emb_size=128,  # Position embedding size, this can be changed
    class_emb_size=128,  # Class embedding size, this can be changed
    stride=1  # Stride for the convolution, this can be changed
)


# Lidara configuration
lidar_config = Config(
    in_channels=81,  # lidar group has 1 channels
    num_patches=1,  # 1 band for Lidar
    kernel_size=1,  # Adjusted to match new patch size
    patch_size=1, # Adjusted to match new patch size
    emb_size=128,  # Embedding size, this can be changed
    dim=128,  # Dimension of the transformer, this can be changed
    depth=6,  # Number of transformer layers, this can be changed
    heads=8,  # Number of attention heads, this can be changed
    dim_head=64,  # Dimension of each attention head, this can be changed
    mlp_dim=256,  # Dimension of the MLP layer, this can be changed
    num_classes=15,  # Number of classes, this can be changed
    dropout=0.4,  # Dropout rate, this can be changed
    pos_emb_size=128,  # Position embedding size, this can be changed
    class_emb_size=128,  # Class embedding size, this can be changed
    stride=1  # Stride for the convolution, this can be changed
)


In [ ]:
# Assume we have loaded one batch of HSI and LiDAR data
hsi_batch = hsi_train  # Shape: (2832, 5, 5, 144)
lidar_batch = lidar_train # Shape: (2832, 5, 5, 1)
print('hsi_batch sahpe before transpose:', hsi_batch.shape)
print('lidar_batch sahpe before transpose:', lidar_batch.shape)
# Transpose the data to have the channel dimension at the correct place
hsi_batch = hsi_batch.transpose(0, 3, 1, 2)  # New shape: (2832, 144, 5, 5)
lidar_batch = lidar_batch.transpose(0, 3, 1, 2)  # New shape: (2832, 1, 5, 5)
print('hsi_batch sahpe after transpose:', hsi_batch.shape)
print('lidar_batch shape after transpose:', lidar_batch.shape)

hsi_batch sahpe before transpose: (4914, 9, 9, 63)
lidar_batch sahpe before transpose: (4914, 9, 9, 1)
hsi_batch sahpe after transpose: (4914, 63, 9, 9)
lidar_batch shape after transpose: (4914, 1, 9, 9)


In [ ]:
# Assume we have loaded one batch of HSI and LiDAR data
one_hsi_batch = hsi_train[:1]  # Shape: (1, 5, 5, 144)
one_lidar_batch = lidar_train[:1]  # Shape: (1, 5, 5, 1)
print('one_hsi_batch sahpe before transpose:', one_hsi_batch.shape)
print('one_lidar_batch sahpe before transpose:', one_lidar_batch.shape)
# Transpose the data to have the channel dimension at the correct place
one_hsi_batch = one_hsi_batch.transpose(0, 3, 1, 2)  # New shape: (1, 144, 5, 5)
one_lidar_batch = one_lidar_batch.transpose(0, 3,1, 2)  # New shape: (1, 1, 5, 5)
print('hsi_batch sahpe after transpose:', one_hsi_batch.shape)
print('lidar_batch shape after transpose:', one_lidar_batch.shape)

one_hsi_batch sahpe before transpose: (1, 9, 9, 63)
one_lidar_batch sahpe before transpose: (1, 9, 9, 1)
hsi_batch sahpe after transpose: (1, 63, 9, 9)
lidar_batch shape after transpose: (1, 1, 9, 9)


In [ ]:
hsi_patch_embedding = PatchEmbedding(hsi_config).to(device)
lidar_patch_embedding = PatchEmbedding(lidar_config).to(device)

# Assume we have loaded one batch of HSI and LiDAR data
one_hsi_batch = hsi_train[:1]  # Shape: (1, 5, 5, 144)
one_lidar_batch = lidar_train[:1]  # Shape: (1, 5, 5, 1)

# Now reshape HSI data such that spatial dimensions (5x5) are flattened and treated as channels
one_hsi_batch_flat = torch.from_numpy(one_hsi_batch.astype(np.float32).reshape(1, hsi_config.in_channels, 63, 1)).to(device)
one_lidar_batch_flat = torch.from_numpy(one_lidar_batch.astype(np.float32).reshape(1, lidar_config.in_channels,1, 1)).to(device)

# Initialize the patch embedding module
hsi_patch_embedding = PatchEmbedding(hsi_config).to(device)
lidar_patch_embedding = PatchEmbedding(lidar_config).to(device)

# Pass the data through the patch embedding module
one_hsi_batch_embedded = hsi_patch_embedding(one_hsi_batch_flat).to(device)
one_lidar_batch_embedded = lidar_patch_embedding(one_lidar_batch_flat).to(device)

print('one_hsi_batch_embedded shape:', one_hsi_batch_embedded.shape)
print('one_lidar_batch_embedded shape:', one_lidar_batch_embedded.shape)

one_hsi_batch_embedded shape: torch.Size([1, 64, 128])
one_lidar_batch_embedded shape: torch.Size([1, 2, 128])


### Intialisation CrossAttention Class

In [ ]:
pip install torch torchvision torchaudio cudatoolkit=11.1 -f https://download.pytorch.org/whl/torch_stable.html


ERROR: Invalid requirement: 'cudatoolkit=11.1'
Hint: = is not a valid operator. Did you mean == ?
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Assume we have loaded one batch of HSI and LiDAR data
hsi_batch_6 = hsi_train[:6]  # Shape: (6, 5, 5, 144)
lidar_batch_6 = lidar_train[:6]  # Shape: (6, 5, 5, 1)

print("hsi_batch_6  shape:", hsi_batch_6 .shape)
print("lidar_batch_6 shape:", lidar_batch_6.shape)

# Now reshape HSI data such that spatial dimensions (5x5) are flattened and treated as channels
hsi_batch_6_flat = torch.from_numpy(hsi_batch_6.astype(np.float32).reshape(6, hsi_config.in_channels, 63, 1))
lidar_batch_6_flat = torch.from_numpy(lidar_batch_6.astype(np.float32).reshape(6, lidar_config.in_channels, 1, 1))

print("hsi_batch_6_flat  shape:", hsi_batch_6_flat .shape)
print("lidar_batch_6_flat shape:", lidar_batch_6_flat.shape)

hsi_batch_6_flat = hsi_batch_6_flat.to(device)  # Move the tensor to GPU
lidar_batch_6_flat = lidar_batch_6_flat.to(device)  # Move the tensor to GPU

hsi_patch_embedding = PatchEmbedding(hsi_config).to(device)
lidar_patch_embedding = PatchEmbedding(lidar_config).to(device)

# Pass the data through the patch embedding module
hsi_batch_6_embedded  = hsi_patch_embedding(hsi_batch_6_flat).to(device)
lidar_batch_6_embedded = lidar_patch_embedding(lidar_batch_6_flat).to(device)

#device = torch.device("cuda:0")  # Define the device (GPU)

# Move the tensors to the desired device
hsi_batch_6_embedded = hsi_batch_6_embedded.to(device)
lidar_batch_6_embedded = lidar_batch_6_embedded.to(device)
print("hsi_batch_6_embedded  shape:", hsi_batch_6_embedded .shape)
print("lidar_batch_6_embedded shape:", lidar_batch_6_embedded.shape)

# Define the dimension of the model and the number of heads
d_model = hsi_config.emb_size  # the output dimension of PatchEmbedding
num_heads = hsi_config.heads  # the number of attention heads in the transformer

# Initialize CrossAttention module
cross_attention = CrossAttention(lidar_config, hsi_config).to(device)

# Apply the cross attention
output,attn_scores = cross_attention(lidar_batch_6_embedded, hsi_batch_6_embedded)


print("Cross attention output shape:", attn_scores.shape)
print("Output shape:", output.shape)

hsi_batch_6  shape: (6, 9, 9, 63)
lidar_batch_6 shape: (6, 9, 9, 1)
hsi_batch_6_flat  shape: torch.Size([6, 81, 63, 1])
lidar_batch_6_flat shape: torch.Size([6, 81, 1, 1])
hsi_batch_6_embedded  shape: torch.Size([6, 64, 128])
lidar_batch_6_embedded shape: torch.Size([6, 2, 128])
Cross attention output shape: torch.Size([6, 63, 63])
Output shape: torch.Size([6, 63, 512])


In [ ]:
# Assume we have loaded one batch of HSI and LiDAR data
hsi_batch_6 = hsi_train[:6]  # Shape: (6, 5, 5, 144)
lidar_batch_6 = lidar_train[:6]  # Shape: (6, 5, 5, 1)

print("hsi_batch_6  shape:", hsi_batch_6 .shape)
print("lidar_batch_6 shape:", lidar_batch_6.shape)

# Now reshape HSI data such that spatial dimensions (5x5) are flattened and treated as channels
#hsi_batch_6_flat = torch.from_numpy(hsi_batch_6.astype(np.float32).reshape(6, 25, 63, 1))
#lidar_batch_6_flat = torch.from_numpy(lidar_batch_6.astype(np.float32).reshape(6, 25, 1, 1))
hsi_batch_6_flat = torch.from_numpy(hsi_batch_6.astype(np.float32).reshape(6, hsi_config.in_channels, hsi_config.num_patches, 1))
lidar_batch_6_flat = torch.from_numpy(lidar_batch_6.astype(np.float32).reshape(6, lidar_config.in_channels, lidar_config.num_patches, 1))

print("hsi_batch_6_flat  shape:", hsi_batch_6_flat .shape)
print("lidar_batch_6_flat shape:", lidar_batch_6_flat.shape)

hsi_batch_6_flat = hsi_batch_6_flat.to(device)  # Move the tensor to GPU
lidar_batch_6_flat = lidar_batch_6_flat.to(device)  # Move the tensor to GPU

hsi_patch_embedding = PatchEmbedding(hsi_config).to(device)
lidar_patch_embedding = PatchEmbedding(lidar_config).to(device)

# Pass the data through the patch embedding module
hsi_batch_6_embedded  = hsi_patch_embedding(hsi_batch_6_flat)
lidar_batch_6_embedded = lidar_patch_embedding(lidar_batch_6_flat)

device = torch.device("cuda:0")  # Define the device (GPU)

# Move the tensors to the desired device
hsi_batch_6_embedded = hsi_batch_6_embedded.to(device)
lidar_batch_6_embedded = lidar_batch_6_embedded.to(device)
print("hsi_batch_6_embedded  shape:", hsi_batch_6_embedded .shape)
print("lidar_batch_6_embedded shape:", lidar_batch_6_embedded.shape)

# Define the dimension of the model and the number of heads
d_model = hsi_config.emb_size  # the output dimension of PatchEmbedding
num_heads = hsi_config.heads  # the number of attention heads in the transformer

# Initialize CrossAttention module
cross_attention = CrossAttention(lidar_config, hsi_config).to(device)

# Apply the cross attention
output,attn_scores = cross_attention(lidar_batch_6_embedded, hsi_batch_6_embedded)


print("Cross attention output shape:", attn_scores.shape)
print("Output shape:", output.shape)




hsi_batch_6  shape: (6, 9, 9, 63)
lidar_batch_6 shape: (6, 9, 9, 1)
hsi_batch_6_flat  shape: torch.Size([6, 81, 63, 1])
lidar_batch_6_flat shape: torch.Size([6, 81, 1, 1])


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
cross_attention

CrossAttention(
  (to_q): Linear(in_features=128, out_features=512, bias=False)
  (to_k): Linear(in_features=128, out_features=512, bias=False)
  (to_v): Linear(in_features=128, out_features=512, bias=False)
  (to_out): Linear(in_features=512, out_features=63, bias=True)
)

In [ ]:
# Create model instance
model = CrossAttentionModel(hsi_config, lidar_config).to(device)
model

CrossAttentionModel(
  (hsi_patch_embedding): PatchEmbedding(
    (proj): Conv2d(81, 128, kernel_size=(1, 1), stride=(1, 1))
  )
  (lidar_patch_embedding): PatchEmbedding(
    (proj): Conv2d(81, 128, kernel_size=(1, 1), stride=(1, 1))
  )
  (cross_attention): CrossAttention(
    (to_q): Linear(in_features=128, out_features=512, bias=False)
    (to_k): Linear(in_features=128, out_features=512, bias=False)
    (to_v): Linear(in_features=128, out_features=512, bias=False)
    (to_out): Linear(in_features=512, out_features=63, bias=True)
  )
)

# 4.1  Training DataLoader for Cross Attention Module

In [ ]:
# from sklearn.model_selection import train_test_split

# # Initialize empty lists to store the train and test samples
# hsi_train = []
# hsi_test = []
# lidar_train = []
# lidar_test = []
# y_train = []
# y_test = []

# # Split the data for each class
# for class_number, class_info in class_dict.items():
#     # Get the indices of the samples for this class
#     class_indices = np.where(labels == class_number)[0]

#     # Shuffle the class indices
#     np.random.shuffle(class_indices)

#     # Split the class indices into training and test indices
#     train_indices = class_indices[:class_info["training"]]
#     test_indices = class_indices[class_info["training"]:class_info["samples"]]

#     # Add the selected training samples to the train lists
#     hsi_train.append(hsi_samples[train_indices])
#     lidar_train.append(lidar_samples[train_indices])
#     y_train.append(labels[train_indices])

#     # Add the remaining samples to the test lists
#     hsi_test.append(hsi_samples[test_indices])
#     lidar_test.append(lidar_samples[test_indices])
#     y_test.append(labels[test_indices])

# # Concatenate the train and test lists to create the train and test arrays
# hsi_train = np.concatenate(hsi_train)
# hsi_test = np.concatenate(hsi_test)
# lidar_train = np.concatenate(lidar_train)
# lidar_test = np.concatenate(lidar_test)
# y_train = np.concatenate(y_train)
# y_test = np.concatenate(y_test)

# # Print the shapes of the train and test arrays
# print('hsi_train shape:', hsi_train.shape)
# print('hsi_test shape:', hsi_test.shape)
# print('lidar_train shape:', lidar_train.shape)
# print('lidar_test shape:', lidar_test.shape)
# print('y_train shape:', y_train.shape)
# print('y_test shape:', y_test.shape)

In [ ]:
class HyperspectralDataset(Dataset):
    def __init__(self, hsi_samples, lidar_samples, labels):
        self.hsi_samples = hsi_samples
        self.lidar_samples = lidar_samples
        self.labels = labels

    def __len__(self):
        return len(self.hsi_samples)

    def __getitem__(self, idx):
        hsi_patch = self.hsi_samples[idx].float().to(device)
        lidar_patch = self.lidar_samples[idx].float().to(device)
        label = self.labels[idx]

        # Convert label to a tensor and reshape to match the model's output shape
        label = torch.tensor(label).repeat(hsi_patch.shape[1], 1).to(device)

        return hsi_patch, lidar_patch, label



In [ ]:
labels_tensor = torch.tensor(y_train)
labels_tensor -= 1  # Shift label values to the range [0, C-1]
print('labels_tensor:',labels_tensor.shape)

labels_tensor: torch.Size([4914, 6])


/tmp/ipykernel_8353/1321332885.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_tensor = torch.tensor(y_train)


In [ ]:
# Assuming labels_tensor is one-hot encoded or contains probabilities
labels_tensor = torch.argmax(labels_tensor, dim=1)
labels_tensor.shape

torch.Size([4914])

In [ ]:
y_train=labels_tensor
print('y_train shape:',y_train.shape)

y_train shape: torch.Size([4914])


In [ ]:

y_test_tensor= torch.tensor(y_test)
y_test_tensor -= 1  # Shift label values to the range [0, C-1]
print('y_test_tensor shape:',y_test_tensor.shape)

y_test_tensor shape: torch.Size([29395, 6])


/tmp/ipykernel_8353/3569926731.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor= torch.tensor(y_test)


In [ ]:
# Assuming labels_tensor is one-hot encoded or contains probabilities
y_test_tensor = torch.argmax(y_test_tensor, dim=1)
print('y_test_tensor shape:',y_test_tensor.shape)

y_test_tensor shape: torch.Size([29395])


In [ ]:
y_test= y_test_tensor

In [ ]:
unique_classes = torch.unique(labels_tensor)
print("Unique classes:", unique_classes)

Unique classes: tensor([0, 1, 2, 3, 4, 5])


In [ ]:
from sklearn.model_selection import train_test_split

#Resahpe the data

# Move the input data to the desired device
# Assume we have loaded one batch of HSI and LiDAR data
# hsi_batch = hsi_train # Shape: (2832, 5, 5, 144)
# lidar_batch= lidar_train# Shape: (2832, 5, 5, 1)
# label_batch=y_train
# print("hsi_batch shape:", hsi_batch.shape)
# print("lidar_batch shape:", lidar_batch.shape)
# print("label_batch shape:", label_batch.shape)

## Reshape the data
hsi_batch_flat = torch.from_numpy(hsi_train.astype(np.float32).reshape(4914, hsi_config.in_channels, 63, 1))
lidar_batch_flat = torch.from_numpy(lidar_train.astype(np.float32).reshape(4914, lidar_config.in_channels, 1, 1))

print("hsi_batch shape:", hsi_batch_flat .shape)
print("lidar_batch shape:", lidar_batch_flat.shape)

hsi_batch_flat = hsi_batch_flat.to(device)  # Move the tensor to GPU
lidar_batch_flat = lidar_batch_flat.to(device)  # Move the tensor to GPU

# # Split into train and test
# hsi_samples_train, hsi_samples_test, lidar_samples_train, lidar_samples_test, labels_train, labels_test = train_test_split(
#     hsi_batch_flat, lidar_batch_flat, label_batch, test_size=0.05, random_state=42)
# print('labels_train shape:',labels_train.shape)

# Split train into train and validation
hsi_samples_train, hsi_samples_val, lidar_samples_train, lidar_samples_val, labels_train, labels_val = train_test_split(
    hsi_batch_flat, lidar_batch_flat, y_train, test_size=0.20, random_state=42)
print('hsi_samples_train shape:',hsi_samples_train.shape)
print('hsi_samples_val shape:',hsi_samples_val.shape)
print('lidar_samples_train shape:',lidar_samples_train.shape)
print('lidar_samples_valshape:',lidar_samples_val.shape)
print('labels_train, shape:',labels_train.shape)
print('labels_val, shape:',labels_val.shape)

# Now test data.
## Reshape the data
hsi_test_batch_flat = torch.from_numpy(hsi_test.astype(np.float32).reshape(29395, hsi_config.in_channels, 63, 1))
lidar_test_batch_flat = torch.from_numpy(lidar_test.astype(np.float32).reshape(29395, lidar_config.in_channels, 1, 1))

# Create Datasets
train_dataset = HyperspectralDataset(hsi_samples_train, lidar_samples_train, labels_train)
val_dataset = HyperspectralDataset(hsi_samples_val, lidar_samples_val, labels_val)
test_dataset = HyperspectralDataset(hsi_test_batch_flat, lidar_test_batch_flat, y_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

hsi_batch shape: torch.Size([4914, 81, 63, 1])
lidar_batch shape: torch.Size([4914, 81, 1, 1])
hsi_samples_train shape: torch.Size([3931, 81, 63, 1])
hsi_samples_val shape: torch.Size([983, 81, 63, 1])
lidar_samples_train shape: torch.Size([3931, 81, 1, 1])
lidar_samples_valshape: torch.Size([983, 81, 1, 1])
labels_train, shape: torch.Size([3931])
labels_val, shape: torch.Size([983])


# 4.2 Training CorssAttentionMode ltraining

In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import copy

# Define the batch size
batch_size = 32

# Create the data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create model instance
model = CrossAttentionModel(hsi_config, lidar_config).to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1.00) #loss 2.3866

# Initialize the best validation loss and best_model_wts before the training loop
best_val_loss = float('inf')  # Initialize with a very high value
best_model_wts = copy.deepcopy(model.state_dict())  # Initialize with the model weights

# Instantiate the loss function
criterion = nn.CrossEntropyLoss()

# Initialize the lowest_loss and best_model_wts before the training loop
lowest_loss = float('inf')  # Initialize with a very high value
best_model_wts = copy.deepcopy(model.state_dict())  # Initialize with the model weights

num_epochs = 200

for epoch in range(num_epochs):
    for batch_idx, (hsi_batch, lidar_batch, label_batch) in enumerate(train_loader):
        # Move tensors to the configured device
        hsi_batch = hsi_batch.to(device)
        lidar_batch = lidar_batch.to(device)
        label_batch = label_batch.to(device)  # Reshape labels
        # print('hsi_batch shape:', hsi_batch.shape)
        # print('lidar_batch:', lidar_batch.shape)
        # print('label_batch shape:', label_batch.shape)

        # Forward pass
        output,attn_scores  = model(lidar_batch, hsi_batch)
        #print('output shape:', output.shape)

        # Calculate loss
        loss = criterion(output.transpose(1, 2), label_batch.squeeze(2))
        #loss = criterion(output, label_batch.long())

        # Print loss every 10 batches
        #if (batch_idx + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


        # If the loss is lower than the current lowest, save the model's state
        if loss.item() < lowest_loss:
            lowest_loss = loss.item()
            best_model_wts = copy.deepcopy(model.state_dict())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Step the learning rate scheduler
        scheduler.step()

# Print finishing training
print('Finishing training')

# Save the best model weights
torch.save(best_model_wts, 'best_model_weights.pth')


/tmp/ipykernel_8353/3897225702.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label = torch.tensor(label).repeat(hsi_patch.shape[1], 1).to(device)


Epoch [1/200], Batch [1/123], Loss: 86.2442
Epoch [1/200], Batch [2/123], Loss: 73.4336
Epoch [1/200], Batch [3/123], Loss: 70.7271
Epoch [1/200], Batch [4/123], Loss: 63.4063
Epoch [1/200], Batch [5/123], Loss: 41.7166
Epoch [1/200], Batch [6/123], Loss: 73.7784
Epoch [1/200], Batch [7/123], Loss: 53.7144
Epoch [1/200], Batch [8/123], Loss: 54.1906
Epoch [1/200], Batch [9/123], Loss: 76.5028
Epoch [1/200], Batch [10/123], Loss: 47.3111
Epoch [1/200], Batch [11/123], Loss: 39.5063
Epoch [1/200], Batch [12/123], Loss: 52.8524
Epoch [1/200], Batch [13/123], Loss: 66.4406
Epoch [1/200], Batch [14/123], Loss: 35.8733
Epoch [1/200], Batch [15/123], Loss: 35.9469
Epoch [1/200], Batch [16/123], Loss: 32.6988
Epoch [1/200], Batch [17/123], Loss: 38.9532
Epoch [1/200], Batch [18/123], Loss: 36.6473
Epoch [1/200], Batch [19/123], Loss: 27.8115
Epoch [1/200], Batch [20/123], Loss: 33.7839
Epoch [1/200], Batch [21/123], Loss: 27.0236
Epoch [1/200], Batch [22/123], Loss: 36.0117
Epoch [1/200], Batc

In [ ]:
# Save the best model state
torch.save(best_model_wts, 'best_model_weights.pth')


In [ ]:
# Load the saved model
best_model = CrossAttentionModel(hsi_config, lidar_config)

# Load the weights
best_model.load_state_dict(torch.load('best_model_weights.pth'))

# Move the model to the GPU
best_model = best_model.to(device)

# Now your model is ready for making predictions on GPU


# 5.0 Load Trained Model

In [ ]:
# Load the model
model = CrossAttentionModel(hsi_config, lidar_config)
model.load_state_dict(torch.load('best_model_weights.pth'))
model.eval()  # set the model to evaluation mode

# Move the model to the GPU
#
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())


Model's state_dict:
hsi_patch_embedding.pos_embedding 	 torch.Size([1, 64, 128])
hsi_patch_embedding.cls_token 	 torch.Size([1, 1, 128])
hsi_patch_embedding.proj.weight 	 torch.Size([128, 1, 1, 1])
hsi_patch_embedding.proj.bias 	 torch.Size([128])
lidar_patch_embedding.pos_embedding 	 torch.Size([1, 2, 128])
lidar_patch_embedding.cls_token 	 torch.Size([1, 1, 128])
lidar_patch_embedding.proj.weight 	 torch.Size([128, 1, 1, 1])
lidar_patch_embedding.proj.bias 	 torch.Size([128])
cross_attention.to_q.weight 	 torch.Size([512, 128])
cross_attention.to_k.weight 	 torch.Size([512, 128])
cross_attention.to_v.weight 	 torch.Size([512, 128])
cross_attention.to_out.weight 	 torch.Size([63, 512])
cross_attention.to_out.bias 	 torch.Size([63])


In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor])


Model's state_dict:
hsi_patch_embedding.pos_embedding 	 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])
hsi_patch_embedding.cls_token 	 tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]])
hsi_patch_embedding.proj.weight 	 tensor([[[[-0.6715]]],


        [[[ 0.63

In [ ]:
# Ensure the model is in evaluation mode
best_model.eval()

# Initialize a list to hold all the predictions and attention scores
all_predictions = []
all_attn_scores = []

# Loop over the validation set
for hsi_batch, lidar_batch, _ in val_loader:  # we don't need the labels for predictions
    # Move the batch to the desired device
    hsi_batch = hsi_batch.to(device)
    lidar_batch = lidar_batch.to(device)

    # Pass the batch through the model
    with torch.no_grad():
        output, attn_scores = best_model(lidar_batch, hsi_batch)

        # Add the predictions and attention scores to our lists
        all_predictions.append(output.cpu().numpy())
        all_attn_scores.append(attn_scores.cpu().numpy())

# Concatenate all predictions and attention scores into a single numpy array
all_predictions = np.concatenate(all_predictions)
all_attn_scores = np.concatenate(all_attn_scores)

# Now you can use the predictions and attention scores as needed
print('all_predictions shape:', all_predictions.shape )
print('all_attn_scores shape:', all_attn_scores.shape )


all_predictions shape: (467, 63, 512)
all_attn_scores shape: (467, 63, 63)


# Band Selection

Calculate the variance of the feature vectors for each band across all samples and then find the bands with the maximum variance

In [ ]:
# First, take the mean across all samples and heads. This will result in a single 144-dim vector.
mean_attention = np.mean(all_attn_scores, axis=(0, 1))
print('mean_attention:',mean_attention.shape)

# Then, sort the bands by attention. This will give you the band indices in descending order of attention.
sorted_band_indices = np.argsort(mean_attention)[::-1]
print('sorted_band_indices :',sorted_band_indices.shape)

# If you want to see the attention values as well, you can sort the mean_attention array in the same order.
sorted_attention_values = mean_attention[sorted_band_indices]
print('sorted_attention_values :',sorted_attention_values.shape)

N = 70  # Top N bands
top_N_bands = sorted_band_indices[:N]
top_N_attention_values = sorted_attention_values[:N]

# Print the top N band indices with their attention scores
print(f"Top {N} bands based on the mean attention score:")
for band_index, attention_value in zip(top_N_bands, top_N_attention_values):
    print(f"Band index: {band_index}, Mean attention score: {attention_value}")

# Print the list of top N band indices
print("\nList of top {} band indices:".format(N))
print(top_N_bands.tolist())


mean_attention: (63,)
sorted_band_indices : (63,)
sorted_attention_values : (63,)
Top 70 bands based on the mean attention score:
Band index: 54, Mean attention score: 65.69277954101562
Band index: 61, Mean attention score: 58.85620880126953
Band index: 9, Mean attention score: 56.95847702026367
Band index: 39, Mean attention score: 54.948387145996094
Band index: 35, Mean attention score: 52.40194320678711
Band index: 59, Mean attention score: 48.99981689453125
Band index: 58, Mean attention score: 48.081111907958984
Band index: 16, Mean attention score: 43.53626251220703
Band index: 47, Mean attention score: 42.674068450927734
Band index: 32, Mean attention score: 42.53934097290039
Band index: 0, Mean attention score: 38.19260025024414
Band index: 31, Mean attention score: 36.499664306640625
Band index: 60, Mean attention score: 33.035743713378906
Band index: 56, Mean attention score: 31.52651596069336
Band index: 17, Mean attention score: 30.98958396911621
Band index: 21, Mean attent

In [ ]:
# Patch_size=1
bands_p1=[54, 61, 9, 39, 35, 59, 58, 16, 47, 32, 0, 31, 60, 56, 17, 21, 8, 23, 49, 19, 36, 25, 15, 38, 29, 6, 43, 27, 52, 13, 42, 1, 45, 12, 5, 46, 7, 33, 34, 50, 30, 51, 20, 18, 14, 53, 26, 22, 28, 11, 24, 3, 57, 37, 10, 2, 4, 48, 41, 40, 44, 62, 55]

In [ ]:
# Using Lidar spatial feature to sorting
bands=[20, 53, 13, 3, 36, 19, 52, 62, 30, 21, 51, 4, 61, 48, 29, 40, 23, 33, 59, 56, 32, 14, 46, 50, 39, 5, 38, 9, 35, 8, 28, 42, 18, 60, 2, 11, 12, 54, 27, 57, 58, 41, 10, 7, 37, 22, 43, 49, 47, 55, 25, 1, 45, 34, 44, 26, 24, 31, 15, 0, 6, 16, 17]

In [ ]:
# patch_size=3 band selction
bands=[34, 57, 33, 50, 20, 40, 35, 31, 39, 13, 26, 53, 51, 27, 47, 61, 6, 58, 9, 5, 46, 54, 4, 16, 43, 12, 19, 36, 1, 30, 2, 28, 7, 32, 42, 25, 8, 44, 17, 23, 38, 52, 15, 14, 21, 56, 48, 49, 0, 62, 10, 45, 59, 22, 37, 24, 60, 41, 18, 11, 3, 55, 29]

In [ ]:
# Patch_size= 5 band selection
bands=[54, 15, 20, 22, 27, 48, 38, 60, 37, 33, 36, 14, 30, 56, 7, 47, 51, 23, 0, 50, 1, 12, 41, 43, 2, 13, 19, 45, 28, 31, 16, 34, 57, 21, 18, 6, 61, 24, 17, 49, 46, 62, 55, 44, 3, 35, 11, 58, 10, 9, 32, 29, 39, 8, 59, 53, 5, 42, 4, 52, 40, 25, 26]

In [ ]:
# Patch_size= 7 band selection
bands=[18, 53, 24, 45, 31, 26, 9, 41, 12, 29, 36, 42, 5, 47, 6, 60, 59, 11, 22, 17, 58, 15, 14, 32, 44, 3, 16, 19, 13, 27, 50, 51, 46, 4, 25, 7, 30, 43, 37, 52, 21, 2, 39, 57, 56, 0, 49, 1, 62, 10, 38, 34, 8, 48, 23, 20, 28, 33, 40, 54, 61, 55, 35]

In [ ]:
# Patch_size= 9 band selection
bands=[37, 52, 39, 50, 46, 42, 31, 35, 18, 34, 43, 26, 14, 25, 59, 30, 44, 22, 57, 21, 5, 32, 6, 58, 27, 38, 36, 4, 49, 8, 56, 20, 12, 55, 0, 61, 11, 16, 54, 53, 3, 17, 51, 7, 33, 29, 41, 9, 24, 40, 1, 10, 23, 45, 47, 15, 28, 2, 13, 62, 48, 60, 19]

In [ ]:
## Patch_size= 11 band selection
bands=[26, 52, 20, 14, 0, 10, 53, 35, 37, 36, 41, 9, 2, 6, 25, 28, 8, 19, 7, 45, 51, 46, 49, 44, 61, 60, 34, 5, 40, 30, 50, 12, 24, 42, 29, 18, 38, 39, 33, 21, 55, 56, 48, 17, 16, 59, 1, 23, 32, 57, 15, 13, 3, 43, 31, 58, 47, 62, 27, 54, 11, 4, 22]